In [336]:
import numpy as np
import pandas as pd
import requests
import math
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
# from sklearn.impute import SimpleImputer
# from tensorflow.keras.preprocessing.text import Tokenizer
# from sklearn.model_selection import GridSearchCV,cross_validate
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
# from sklearn.naive_bayes import MultinomialNB
import nltk
from google.cloud import storage

In [337]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pyfenix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pyfenix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pyfenix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [338]:
def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

# Apply to all texts
# data['clean_text'] = data.text.apply(clean)
# data['clean_text'] = data['clean_text'].astype('str')

## Testing bigger df


In [339]:
big_df= pd.read_csv("../raw_data/bigger_df/steam_data.csv")
big_df.head()

,url,name,categories,img_url,user_reviews,all_reviews,date,developer,publisher,price,pegi,pegi_url
0,https://store.steampowered.com/app/945360/Amon...,Among Us,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Overwhelmingly Positive(151,281)- 96% of the 1...","Overwhelmingly Positive(224,878)- 95% of the 2...","Nov 16, 2018",Innersloth,Innersloth,Buy Among Us$4.99Add to Cart,-,-
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,Steam AchievementsFull controller supportSteam...,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Very Positive(90,780)- 88% of the 90,780 user ...","Very Positive(4,843,904)- 87% of the 4,843,904...","Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,Play Counter-Strike: Global OffensiveFree to P...,-,-
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,MMOOnline PvPOnline Co-opSteam AchievementsFul...,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Very Positive(32,436)- 84% of the 32,436 user ...","Very Positive(223,706)- 80% of the 223,706 use...","Aug 3, 2020",Mediatonic,Devolver Digital,Buy Fall Guys$19.99Add to Cart,-,-
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,Single-playerOnline PvPSteam AchievementsSteam...,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Very Positive(5,359)- 91% of the 5,359 user re...","Very Positive(18,951)- 92% of the 18,951 user ...","Sep 1, 2020",Paradox Development Studio,Paradox Interactive,Buy Crusader Kings III$49.99Add to Cart,BloodLanguagePartial NuditySexual ThemesUse of...,https://steamstore-a.akamaihd.net/public/share...
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,Single-playerOnline PvPOnline Co-opSteam Achie...,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Very Positive(9,147)- 87% of the 9,147 user re...","Very Positive(284,689)- 86% of the 284,689 use...","Oct 1, 2019",Bungie,Bungie,Play Destiny 2Free To PlayPlay Game,Blood Language Violence,https://steamstore-a.akamaihd.net/public/share...


In [340]:
big_df['all_reviews'][3]

'Very Positive(18,951)- 92% of the 18,951 user reviews for this game are positive.'

In [341]:
big_df.isnull().sum()

url                0
name               5
categories       595
img_url            0
user_reviews       0
all_reviews        2
date               2
developer          2
publisher          0
price              0
pegi            4283
pegi_url           0
dtype: int64

In [342]:
big_df.dtypes

url             object
name            object
categories      object
img_url         object
user_reviews    object
all_reviews     object
date            object
developer       object
publisher       object
price           object
pegi            object
pegi_url        object
dtype: object

In [343]:
content_df = pd.read_csv('../raw_data/bigger_df/text_content.csv', index_col='url')
content_df.head()

,desc,full_desc,requirements,popu_tags
url,,,,
https://store.steampowered.com/app/945360/Among_Us/?snr=1_7_7_230_150_1,An online and local party game of teamwork and...,About This GamePlay with 4-10 player online or...,System RequirementsMinimum:OS: Windows 7 SP1+P...,MultiplayerOnline Co-OpSpaceSocial DeductionSu...
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1,Counter-Strike: Global Offensive (CS: GO) expa...,About This GameCounter-Strike: Global Offensiv...,System RequirementsWindowsMac OS XSteamOS + Li...,FPSShooterMultiplayerCompetitiveActionTeam-Bas...
https://store.steampowered.com/app/1097150/Fall_Guys_Ultimate_Knockout/?snr=1_7_7_230_150_1,Fall Guys is a massively multiplayer party gam...,About This GameFall Guys: Ultimate Knockout fl...,System RequirementsMinimum:Requires a 64-bit p...,MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
https://store.steampowered.com/app/1158310/Crusader_Kings_III/?snr=1_7_7_230_150_1,Paradox Development Studio brings you the sequ...,About This GameParadox Development Studio brin...,System RequirementsWindowsMac OS XSteamOS + Li...,StrategyRPGSimulationMedievalGrand StrategyHis...
https://store.steampowered.com/app/1085660/Destiny_2/?snr=1_7_7_230_150_1,Destiny 2 is an action MMO with a single evolv...,About This GameDive into the world of Destiny ...,System RequirementsMinimum:Requires a 64-bit p...,Free to PlayLooter ShooterMultiplayerFPSAction...


In [344]:
content_df['popu_tags']

url
https://store.steampowered.com/app/945360/Among_Us/?snr=1_7_7_230_150_1                                                      MultiplayerOnline Co-OpSpaceSocial DeductionSu...
https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/?snr=1_7_7_230_150_1                                   FPSShooterMultiplayerCompetitiveActionTeam-Bas...
https://store.steampowered.com/app/1097150/Fall_Guys_Ultimate_Knockout/?snr=1_7_7_230_150_1                                  MultiplayerFunnyBattle RoyaleOnline Co-OpFamil...
https://store.steampowered.com/app/1158310/Crusader_Kings_III/?snr=1_7_7_230_150_1                                           StrategyRPGSimulationMedievalGrand StrategyHis...
https://store.steampowered.com/app/1085660/Destiny_2/?snr=1_7_7_230_150_1                                                    Free to PlayLooter ShooterMultiplayerFPSAction...
                                                                                                                         

In [345]:
json_df= pd.read_json('../raw_data/bigger_df/final_data_new.json')
json_df.head()

,img_url,date,developer,publisher,full_desc,requirements,popu_tags,price,url_info,name,categories,pegi
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,"{'sort': 'game', 'desc': 'About This Game Play...",{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,{'url': 'https://store.steampowered.com/app/94...,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,"{'sort': 'game', 'desc': 'About This Game Coun...",{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,{'url': 'https://store.steampowered.com/app/73...,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,"{'sort': 'game', 'desc': 'About This Game Fall...",{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,{'url': 'https://store.steampowered.com/app/10...,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,"{'sort': 'game', 'desc': 'About This Game Para...",{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,{'url': 'https://store.steampowered.com/app/11...,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,"{'sort': 'game', 'desc': 'About This Game Dive...",{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,{'url': 'https://store.steampowered.com/app/10...,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...


In [346]:
json_df['requirements'][35487]

{'minimum': {'windows': {'processor': ' Intel® Core™ i5-2550 or AMD FX-6300',
   'memory': ' 8 GB RAM',
   'graphics': ' Radeon™ RX 460 or NVIDIA® GeForce® GTX 1050',
   'os': ' Windows® 10 64-bit'}},
 'recommended': {'windows': {'processor': ' Intel® Core™ i7-6700K or AMD Ryzen™ 5 1400',
   'memory': ' 8 GB RAM',
   'graphics': ' Radeon™ RX 470 or NVIDIA® GeForce® GTX 1060',
   'os': 'Windows® 10 64-bit'}}}

In [347]:
json_df = json_df.drop(columns='full_desc').assign(**json_df.full_desc.dropna().apply(pd.Series))

In [348]:
json_df['url_info'][0]

{'url': 'https://store.steampowered.com/app/945360/Among_Us/?snr=1_7_7_230_150_1',
 'id': '945360',
 'type': 'app',
 'url_name': 'Among Us'}

In [349]:
json_df = json_df.drop(columns='url_info').assign(**json_df.url_info.dropna().apply(pd.Series))

In [350]:
json_df.head()

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN,game,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,945360,app,Among Us
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN,game,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,730,app,CounterStrike Global Offensive
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN,game,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,1097150,app,Fall Guys Ultimate Knockout
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,1158310,app,Crusader Kings III
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,1085660,app,Destiny 2


In [351]:
json_df['type'].isnull().value_counts()

False    68718
True      6103
Name: type, dtype: int64

In [352]:
json_df[json_df['type'].isnull()]

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
56,https://steamcdn-a.akamaihd.net/steam/apps/252...,"Jul 7, 2015",Psyonix LLC,Psyonix LLC,{'minimum': {'windows': {'processor': ' 2.5 GH...,"[Multiplayer, Soccer, Competitive, Sports, Rac...",NaN,Rocket League®,"[Single-playerOnline PvPShared/Split, Screen P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Rocket League is a high-powere...,NaN,NaN,NaN,NaN
106,https://steamcdn-a.akamaihd.net/steam/apps/252...,"Jul 7, 2015",Psyonix LLC,Psyonix LLC,{'minimum': {'windows': {'processor': ' 2.5 GH...,"[Multiplayer, Soccer, Competitive, Sports, Rac...",NaN,Rocket League®,"[Single-playerOnline PvPShared/Split, Screen P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Rocket League is a high-powere...,NaN,NaN,NaN,NaN
538,https://steamcdn-a.akamaihd.net/steam/apps/261...,"Mar 11, 2015",Moon Studios GmbH,Xbox Game Studios,{'minimum': {'windows': {'processor': ' Intel ...,"[Great, Soundtrack, Atmospheric, Platformer, A...",NaN,Ori and the Blind Forest,"[Single-playerSteam Achievements Full, control...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game The forest of Nibel is dying. ...,NaN,NaN,NaN,NaN
619,https://steamcdn-a.akamaihd.net/steam/apps/134...,"Jul 21, 2020","PUBG Corporation, PUBG Corporation",NaN,{'minimum': {'windows': {'processor': ' Intel ...,"[Action, Adventure, Massively, Multiplayer+]",NaN,Survivor Pass: Payback,"[Online PvPDownloadable, Content StatsRemote P...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,content,About This Content Welcome back to the Jungle!...,NaN,NaN,NaN,NaN
639,https://steamcdn-a.akamaihd.net/steam/apps/901...,NaN,"Rockstar North, Rockstar Toronto",NaN,{},"[Open, World, Action, Bowling, Automobile, Mul...",NaN,Grand Theft Auto IV: Complete Edition,"[Single-playerPartial Controller, Support Inco...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game PLEASE NOTE: Microsoft no lon...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74812,https://steamcdn-a.akamaihd.net/steam/apps/134...,"29 Oct, 2020",Never Knows Best,NaN,{},"[Dating, Casual, Visual, Novel, Simulation, Wo...",NaN,恋爱绮谭~不存在的夏天~,"[Single-playerSteam is, learning about, this g...",NaN,game,About This Game FEATURE20WCGL2DSTAFFNEVER KNOW...,NaN,NaN,NaN,NaN
74814,https://steamcdn-a.akamaihd.net/steam/apps/134...,"30 Jun, 2020",zomgbab,NaN,"{'minimum': {'windows': {'processor': ' TBD', ...","[Simulation, Casual, Dark, Humor, Survival, Ho...",NaN,Social Distancing Simulator,"[Single-playerSteam Achievements StatsSteam, i...",NaN,game,About This Game Social distancing is in full e...,NaN,NaN,NaN,NaN
74817,https://steamcdn-a.akamaihd.net/steam/apps/134...,Late 2020,Lychee Game Labs,NaN,{'minimum': {'windows': {'processor': ' Core i...,"[Early, Access, Action, Roguelike, Roguevania,...",NaN,Patch Quest,"[Single-playerShared/Split Screen, Co-opFull c...",NaN,game,About This Game Patch Quest is the only action...,NaN,NaN,NaN,NaN
74818,https://steamcdn-a.akamaihd.net/steam/apps/134...,"6 Nov, 2020",七彩绘色游戏,NaN,{'minimum': {'linux': {'processor': ' Intel酷睿i...,"[Casual, Visual, Novel, Shooter, Nudity, Relax...",NaN,球球少女,"[Single-playerSteam is, learning about, this g...",NaN,game,About This Game GALGAME99CG,NaN,NaN,NaN,NaN


In [353]:
json_df['sort'].value_counts()

game       48014
content    25261
            1414
Name: sort, dtype: int64

In [354]:
clean_df = json_df[(json_df['sort']=='game') & (json_df['url'].isnull()!= True)].copy()
clean_df.head()

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,"Nov 16, 2018",Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",NaN,game,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,945360,app,Among Us
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Aug 21, 2012","Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",NaN,game,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,730,app,CounterStrike Global Offensive
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,"Aug 3, 2020",Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",NaN,game,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,1097150,app,Fall Guys Ultimate Knockout
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,"Sep 1, 2020",Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,1158310,app,Crusader Kings III
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,"Oct 1, 2019",Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,game,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,1085660,app,Destiny 2


In [355]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42640 non-null  object
 3   publisher     4511 non-null   object
 4   requirements  42679 non-null  object
 5   popu_tags     42698 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42665 non-null  object
 9   pegi          3396 non-null   object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [356]:
clean_df[['categories', 'popu_tags']]=clean_df[['categories', 'popu_tags']].fillna("game")

In [357]:
clean_df[clean_df['categories'].isnull()]

,img_url,date,developer,publisher,requirements,popu_tags,price,name,categories,pegi,sort,desc,url,id,type,url_name


In [358]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42640 non-null  object
 3   publisher     4511 non-null   object
 4   requirements  42679 non-null  object
 5   popu_tags     42720 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42720 non-null  object
 9   pegi          3396 non-null   object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [359]:
clean_df[['developer', 'requirements', 'pegi','publisher']]=clean_df[['developer', 'requirements', 'pegi','publisher']].fillna("no info")

In [360]:
clean_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42720 entries, 0 to 74820
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   img_url       42720 non-null  object
 1   date          42667 non-null  object
 2   developer     42720 non-null  object
 3   publisher     42720 non-null  object
 4   requirements  42720 non-null  object
 5   popu_tags     42720 non-null  object
 6   price         42720 non-null  object
 7   name          42720 non-null  object
 8   categories    42720 non-null  object
 9   pegi          42720 non-null  object
 10  sort          42720 non-null  object
 11  desc          42720 non-null  object
 12  url           42720 non-null  object
 13  id            42720 non-null  object
 14  type          42720 non-null  object
 15  url_name      42720 non-null  object
dtypes: object(16)
memory usage: 5.5+ MB


In [361]:
clean_df.columns

Index(['img_url', 'date', 'developer', 'publisher', 'requirements',
       'popu_tags', 'price', 'name', 'categories', 'pegi', 'sort', 'desc',
       'url', 'id', 'type', 'url_name'],
      dtype='object')

In [362]:
clean_df=clean_df.drop(columns=['date','sort','id','type','url_name','date'])
clean_df.head()

,img_url,developer,publisher,requirements,popu_tags,price,name,categories,pegi,desc,url
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...


In [363]:
" ".join(clean_df['popu_tags'][0])

'Multiplayer Online Space Social Deduction Survival Funny2 Psychological Local Multiplayer Cartoony Casual Indie Colorful Minigames Local Down Aliens+'

In [364]:

clean_df['cat']=clean_df['categories'].apply(lambda x: " ".join(x))
clean_df.head()

,img_url,developer,publisher,requirements,popu_tags,price,name,categories,pegi,desc,url,cat
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...


In [365]:
clean_df['tags']=clean_df['popu_tags'].apply(lambda x: " ".join(x))
clean_df.head()

,img_url,developer,publisher,requirements,popu_tags,price,name,categories,pegi,desc,url,cat,tags
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,"[Multiplayer, Online, Space, Social, Deduction...",499,Among Us,"[Online PvPLAN, PvPOnline Co-opLAN, Co-opCross...",no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,Multiplayer Online Space Social Deduction Surv...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,"[Shooter, Multiplayer, Competitive, Action, Te...",free,Counter-Strike: Global Offensive,"[Steam Achievements Full, controller supportSt...",no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...,Shooter Multiplayer Competitive Action Team- B...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},"[Multiplayer, Funny, Battle, Royale, Online, F...",199,Fall Guys: Ultimate Knockout,"[MMOOnline PvPOnline, Co-opSteam Achievements ...",no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...,Multiplayer Funny Battle Royale Online Family ...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},"[Strategy, Simulation, Medieval, Grand, Strate...",499,Crusader Kings III,"[Single-playerOnline PvPSteam, Achievements St...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...,Strategy Simulation Medieval Grand Strategy Hi...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},"[Free, Play, Looter, Shooter, Multiplayer, Act...",free,Destiny 2,"[Single-playerOnline PvPOnline, Co-opSteam Ach...",{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...,Free Play Looter Shooter Multiplayer Action On...


In [366]:
clean_df['metadata'] = clean_df[['tags', 'cat']].apply(lambda x: ' '.join(x), axis = 1)


clean_df['metadata'] = clean_df['metadata'].apply(
    lambda x: ' '.join(list(set(x.split())))
)

In [367]:
clean_df['metadata'][0]

'Co-opLAN Indie Play on Tablet Space Psychological Deduction PvPOnline Survival Social PvPLAN Multiplayer Down Co-opCross-Platform Colorful Phone Funny2 Remote Casual Minigames Online Aliens+ Local Cartoony'

In [368]:
metadata_df=clean_df.drop(columns=['popu_tags','categories','tags']).copy()

In [369]:
def clean_text(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    
    return " ".join(lemmatized)

In [370]:
metadata_df.shape

(42720, 11)

In [371]:
metadata_df = metadata_df[
        metadata_df['desc']\
                .str.contains('About This Content', case=False) == False]


In [372]:
metadata_df.shape

(42719, 11)

In [373]:
metadata_df = metadata_df[
        metadata_df['desc']\
                .str.contains('About This Hardware', case=False) == False]

In [374]:
metadata_df.shape

(42719, 11)

In [375]:
metadata_df['clean_description'] = metadata_df.desc.astype('str').apply(clean_text)
metadata_df.head()

,img_url,developer,publisher,requirements,price,name,pegi,desc,url,cat,metadata,clean_description
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,499,Among Us,no info,About This Game Play with 4-10 player online o...,https://store.steampowered.com/app/945360/Amon...,Online PvPLAN PvPOnline Co-opLAN Co-opCross-Pl...,Co-opLAN Indie Play on Tablet Space Psychologi...,game play player online via local wifi attempt...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,free,Counter-Strike: Global Offensive,no info,About This Game Counter-Strike: Global Offensi...,https://store.steampowered.com/app/730/Counter...,Steam Achievements Full controller supportStea...,enabledStats Play on controller Realistic Work...,game counter strike global offensive c go expa...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},199,Fall Guys: Ultimate Knockout,no info,About This Game Fall Guys: Ultimate Knockout f...,https://store.steampowered.com/app/1097150/Fal...,MMOOnline PvPOnline Co-opSteam Achievements Fu...,Battle Indie Play on controller Family Massive...,game fall guy ultimate knockout fling horde co...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},499,Crusader Kings III,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Paradox Development Studio bri...,https://store.steampowered.com/app/1158310/Cru...,Single-playerOnline PvPSteam Achievements Stea...,Customization Workshop Dating Time Sandbox Car...,game paradox development studio brings sequel ...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},free,Destiny 2,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,About This Game Dive into the world of Destiny...,https://store.steampowered.com/app/1085660/Des...,Single-playerOnline PvPOnline Co-opSteam Achie...,Play on controller agreement Tablet Space Firs...,game dive world destiny explore mystery solar ...


In [376]:
metadata_df=metadata_df.drop(columns=['desc','cat'])
metadata_df.head()

,img_url,developer,publisher,requirements,price,name,pegi,url,metadata,clean_description
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,499,Among Us,no info,https://store.steampowered.com/app/945360/Amon...,Co-opLAN Indie Play on Tablet Space Psychologi...,game play player online via local wifi attempt...
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,free,Counter-Strike: Global Offensive,no info,https://store.steampowered.com/app/730/Counter...,enabledStats Play on controller Realistic Work...,game counter strike global offensive c go expa...
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},199,Fall Guys: Ultimate Knockout,no info,https://store.steampowered.com/app/1097150/Fal...,Battle Indie Play on controller Family Massive...,game fall guy ultimate knockout fling horde co...
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},499,Crusader Kings III,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,https://store.steampowered.com/app/1158310/Cru...,Customization Workshop Dating Time Sandbox Car...,game paradox development studio brings sequel ...
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},free,Destiny 2,{'pegi_url': 'https://steamstore-a.akamaihd.ne...,https://store.steampowered.com/app/1085660/Des...,Play on controller agreement Tablet Space Firs...,game dive world destiny explore mystery solar ...


In [377]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42719 entries, 0 to 74820
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   img_url            42719 non-null  object
 1   developer          42719 non-null  object
 2   publisher          42719 non-null  object
 3   requirements       42719 non-null  object
 4   price              42719 non-null  object
 5   name               42719 non-null  object
 6   pegi               42719 non-null  object
 7   url                42719 non-null  object
 8   metadata           42719 non-null  object
 9   clean_description  42719 non-null  object
dtypes: object(10)
memory usage: 3.6+ MB


In [378]:
metadata_df = metadata_df.drop(columns='pegi').assign(**json_df.pegi.dropna().apply(pd.Series))

In [379]:
metadata_df.head()

,img_url,developer,publisher,requirements,price,name,url,metadata,clean_description,pegi_url,pegi_tags,age_limit
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,499,Among Us,https://store.steampowered.com/app/945360/Amon...,Co-opLAN Indie Play on Tablet Space Psychologi...,game play player online via local wifi attempt...,NaN,NaN,NaN
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,free,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,enabledStats Play on controller Realistic Work...,game counter strike global offensive c go expa...,NaN,NaN,NaN
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},199,Fall Guys: Ultimate Knockout,https://store.steampowered.com/app/1097150/Fal...,Battle Indie Play on controller Family Massive...,game fall guy ultimate knockout fling horde co...,NaN,NaN,NaN
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},499,Crusader Kings III,https://store.steampowered.com/app/1158310/Cru...,Customization Workshop Dating Time Sandbox Car...,game paradox development studio brings sequel ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, LanguagePartial, Nudity, Sexual, Theme...",m
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},free,Destiny 2,https://store.steampowered.com/app/1085660/Des...,Play on controller agreement Tablet Space Firs...,game dive world destiny explore mystery solar ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, Language, Violence]",m


In [380]:
metadata_df= metadata_df.assign(mature_encoded=lambda x: x.age_limit.isna().replace((True,False),(0,1)))
metadata_df.head()

,img_url,developer,publisher,requirements,price,name,url,metadata,clean_description,pegi_url,pegi_tags,age_limit,mature_encoded
0,https://steamcdn-a.akamaihd.net/steam/apps/945...,Innersloth,Innersloth,{'minimum': {'windows': {'processor': ' SSE2 i...,499,Among Us,https://store.steampowered.com/app/945360/Amon...,Co-opLAN Indie Play on Tablet Space Psychologi...,game play player online via local wifi attempt...,NaN,NaN,NaN,0
1,https://steamcdn-a.akamaihd.net/steam/apps/730...,"Valve, Hidden Path Entertainment",Valve,{'minimum': {'windows': {'processor': ' Intel®...,free,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...,enabledStats Play on controller Realistic Work...,game counter strike global offensive c go expa...,NaN,NaN,NaN,0
2,https://steamcdn-a.akamaihd.net/steam/apps/109...,Mediatonic,Devolver Digital,{},199,Fall Guys: Ultimate Knockout,https://store.steampowered.com/app/1097150/Fal...,Battle Indie Play on controller Family Massive...,game fall guy ultimate knockout fling horde co...,NaN,NaN,NaN,0
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},499,Crusader Kings III,https://store.steampowered.com/app/1158310/Cru...,Customization Workshop Dating Time Sandbox Car...,game paradox development studio brings sequel ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, LanguagePartial, Nudity, Sexual, Theme...",m,1
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},free,Destiny 2,https://store.steampowered.com/app/1085660/Des...,Play on controller agreement Tablet Space Firs...,game dive world destiny explore mystery solar ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, Language, Violence]",m,1


In [381]:
metadata_df['age_limit'].value_counts()

m    3395
Name: age_limit, dtype: int64

In [382]:
metadata_df[metadata_df['pegi_tags'].isna()==False]

,img_url,developer,publisher,requirements,price,name,url,metadata,clean_description,pegi_url,pegi_tags,age_limit,mature_encoded
3,https://steamcdn-a.akamaihd.net/steam/apps/115...,Paradox Development Studio,Paradox Interactive,{},499,Crusader Kings III,https://store.steampowered.com/app/1158310/Cru...,Customization Workshop Dating Time Sandbox Car...,game paradox development studio brings sequel ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, LanguagePartial, Nudity, Sexual, Theme...",m,1
4,https://steamcdn-a.akamaihd.net/steam/apps/108...,Bungie,Bungie,{},free,Destiny 2,https://store.steampowered.com/app/1085660/Des...,Play on controller agreement Tablet Space Firs...,game dive world destiny explore mystery solar ...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, Language, Violence]",m,1
6,https://steamcdn-a.akamaihd.net/steam/apps/117...,Rare Ltd,Xbox Game Studios,{},399,Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,Exploration Play on Requires Tablet Support Fi...,game gamesea thief offer essential pirate expe...,https://steamstore-a.akamaihd.net/public/share...,"[Violence, Use, of, Alcohol, Crude, Humor]",m,1
7,https://steamcdn-a.akamaihd.net/steam/apps/359...,Ubisoft Montreal,Ubisoft,{'minimum': {'windows': {'processor': ' Intel ...,199,Tom Clancy's Rainbow Six® Siege,https://store.steampowered.com/app/359550/Tom_...,Play on Requires Realistic agreement Tablet Si...,game master art destruction gadgetry tom clanc...,https://steamstore-a.akamaihd.net/public/share...,"[Blood, Drug, Reference, Strong, Language, Vio...",m,1
9,https://steamcdn-a.akamaihd.net/steam/apps/230...,Digital Extremes,Digital Extremes,{'minimum': {'windows': {'processor': 'Intel C...,free,Warframe,https://store.steampowered.com/app/230410/Warf...,Play on Requires Tablet Space Support Ninja 3r...,game choose warframewith warframes fully custo...,https://steamstore-a.akamaihd.net/public/share...,"[Violence,, Blood]",m,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74273,https://steamcdn-a.akamaihd.net/steam/apps/100...,WhaleFood Games,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,KungFu Kickball,https://store.steampowered.com/app/1004620/Kun...,Indie about Play controller Family Leaderboard...,game kungfu kickball kungfu kickball fighting ...,https://steamstore-a.akamaihd.net/public/share...,[Horror],m,1
74350,https://steamcdn-a.akamaihd.net/steam/apps/360...,"Hangar 13, Aspyr (Mac)",2K,{},free,Mafia III: Definitive Edition,https://store.steampowered.com/app/360430/Mafi...,Play on controller agreement Mature Tablet Sto...,game mafia iii definitive edition includes mai...,https://steamstore-a.akamaihd.net/public/share...,"[Sex, ViolenceTobacco, &, Alcohol, DrugsImprop...",m,1
74403,https://steamcdn-a.akamaihd.net/steam/apps/899...,3DClouds,SOEDESCO,{},318,Xenon Racer,https://store.steampowered.com/app/899390/Xeno...,Indie Play on controller Family First- Story C...,game xenon drive futurein near future official...,https://steamstore-a.akamaihd.net/public/share...,[Violence],m,1
74603,https://steamcdn-a.akamaihd.net/steam/apps/739...,The Molasses Flood,no info,{},438,Drake Hollow,https://store.steampowered.com/app/739650/Drak...,Exploration Agriculture Customization controll...,game welcome hollow blighted dying place home ...,https://steamstore-a.akamaihd.net/public/share...,[Horror],m,1


In [383]:
metadata_df['pegi_tags']=metadata_df['pegi_tags'].fillna("no_mature_content")

In [384]:
metadata_df['pegi_url'][3]


'https://steamstore-a.akamaihd.net/public/shared/images/game_ratings/ESRB/m.png'

In [385]:
pegi_tags=[]
for index, content in metadata_df.iterrows():
    if content['pegi_tags']!="no_mature_content":
        pegi_tags += content['pegi_tags'] 

list(set(pegi_tags))

['',
 'Humour,',
 'goreLanguage•',
 'Mature',
 'LanguageAlcohol',
 'Anti-social',
 'language',
 'Langiage',
 'violenceUse',
 "It's",
 'little',
 'everyone',
 'LanguageLyrics',
 'splats',
 'Purchases',
 'by',
 'Cartoonish',
 'defense,',
 'Partial',
 'Language,Interactive',
 'gameplay.',
 '暴力,不當言語',
 'or',
 'Interact,',
 'Language,',
 'other',
 'Assistance',
 'General',
 'up.',
 'BloodUsers',
 'LOOKING',
 'Mention',
 'contains',
 'This',
 'for',
 'BloodFantasy',
 'May',
 'unrealistic',
 '(blood',
 'Violence.Crude',
 'Rating-16-More',
 'Use',
 'Characteristics',
 'image',
 'sexual',
 'In-game',
 'mild',
 'Offensive',
 '(with',
 'blood',
 'esrb.org',
 'from',
 'Simulated',
 'G',
 'Violence)',
 'capture',
 'AlcoholSexual',
 'Dead',
 'GoreAnimated',
 'trading',
 'reference,',
 'images',
 'Anti-Social',
 'imagerySudden',
 'Moderate',
 'Tobacco',
 'Everyone',
 'Frequent',
 'emergencies.',
 'Swords',
 'disabled',
 'ViolenceUsers',
 'Blood,Mild',
 'mature',
 'NudityMature',
 'use',
 'Random',
 '

In [386]:
smaller_df=pd.read_csv("../raw_data/clean_df.csv")
smaller_df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata,cluster
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,co fps meme classic great sci difficult soundt...,29
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,third co fps massively based battle difficult ...,28
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,rich story great space based sci difficult sou...,3
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,sandbox co open fps massively apocalyptic op a...,38
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,sandbox open space massively sci difficult cap...,44


In [387]:
big_df.columns

Index(['url', 'name', 'categories', 'img_url', 'user_reviews', 'all_reviews',
       'date', 'developer', 'publisher', 'price', 'pegi', 'pegi_url'],
      dtype='object')

In [388]:
metadata_df['url']

0        https://store.steampowered.com/app/945360/Amon...
1        https://store.steampowered.com/app/730/Counter...
2        https://store.steampowered.com/app/1097150/Fal...
3        https://store.steampowered.com/app/1158310/Cru...
4        https://store.steampowered.com/app/1085660/Des...
                               ...                        
74799    https://store.steampowered.com/app/1338020/Sup...
74805    https://store.steampowered.com/app/1341230/Sup...
74809    https://store.steampowered.com/app/1344740/FOO...
74813    https://store.steampowered.com/app/1346070/Sig...
74820    https://store.steampowered.com/app/597700/OVIV...
Name: url, Length: 42719, dtype: object

In [389]:
metadata_df=metadata_df.set_index('url').join(big_df[['url','all_reviews']].set_index('url'))
metadata_df.reset_index().head()

,url,img_url,developer,publisher,requirements,price,name,metadata,clean_description,pegi_url,pegi_tags,age_limit,mature_encoded,all_reviews
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,Valve,{},999,Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,no_mature_content,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94..."
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,no info,{'minimum': {'windows': {'processor': ' Intel ...,490,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,no_mature_content,NaN,0,"16 Oct, 2020"


In [390]:
metadata_df.reset_index().head()

,url,img_url,developer,publisher,requirements,price,name,metadata,clean_description,pegi_url,pegi_tags,age_limit,mature_encoded,all_reviews
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,Valve,{},999,Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,no_mature_content,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94..."
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,no info,{'minimum': {'windows': {'processor': ' Intel ...,490,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,no_mature_content,NaN,0,"16 Oct, 2020"


In [391]:
metadata_df['all_reviews'][0]

'Overwhelmingly Positive(94,680)- 96% of the 94,680 user reviews for this game are positive.'

In [392]:
metadata_df['all_reviews'].value_counts()

2020                                                                                 100
2021                                                                                  96
Coming Soon                                                                           68
15 Oct, 2020                                                                          55
Jul 31, 2020                                                                          54
                                                                                    ... 
Very Positive(2,351)- 87% of the 2,351 user reviews for this game are positive.        1
Nov 4, 2011                                                                            1
Very Positive(679)- 83% of the 679 user reviews for this game are positive.            1
Very Positive(742)- 88% of the 742 user reviews for this game are positive.            1
Mostly Positive(1,224)- 77% of the 1,224 user reviews for this game are positive.      1
Name: all_reviews, Le

In [393]:
def clean_review(review):
    if review:
        return review.split('(')[0]
    else:
        return float('nan')

In [394]:
metadata_df['clean_review'] = metadata_df['all_reviews'].apply(lambda x: clean_review(x))

In [395]:
metadata_df['clean_review'].value_counts()

Very Positive              2775
Mostly Positive             761
Overwhelmingly Positive     598
Mixed                       506
2020                        100
                           ... 
24 Apr, 2016                  1
Dec 31, 2006                  1
Jun 25, 2004                  1
Mar 8, 2010                   1
Nov 10, 2010                  1
Name: clean_review, Length: 6077, dtype: int64

In [396]:
metadata_df[(metadata_df['all_reviews'].str.contains("Overwhelmingly Negative|Very Negative|Negative|Mostly Negative|Mixed|Mostly Positive|Positive|Very Positive|Overwhelmingly Positive")==False)].shape

(38303, 14)

In [398]:
metadata_df=metadata_df.reset_index()
metadata_df.head()

,url,img_url,developer,publisher,requirements,price,name,metadata,clean_description,pegi_url,pegi_tags,age_limit,mature_encoded,all_reviews,clean_review
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,Valve,{},999,Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,no_mature_content,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94...",Overwhelmingly Positive
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,no info,{'minimum': {'windows': {'processor': ' Intel ...,free,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,no_mature_content,NaN,0,Winter 2020,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,no info,{'minimum': {'windows': {'processor': ' Intel ...,490,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,no_mature_content,NaN,0,"16 Oct, 2020","16 Oct, 2020"


In [399]:
metadata_df.shape

(42993, 15)

In [419]:
re_df=metadata_df[metadata_df['all_reviews'].str.contains("Overwhelmingly Negative|Very Negative|Negative|Mostly Negative|Mixed|Mostly Positive|Positive|Very Positive|Overwhelmingly Positive")]

re_df.shape

(4690, 15)

In [400]:
reviews_df=metadata_df.drop(columns=['publisher','price','age_limit','pegi_tags'])

In [401]:
reviews_df.head()

,url,img_url,developer,requirements,name,metadata,clean_description,pegi_url,mature_encoded,all_reviews,clean_review
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,{},Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94...",Overwhelmingly Positive
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,{'minimum': {'windows': {'processor': ' Intel ...,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,0,"16 Oct, 2020","16 Oct, 2020"


In [402]:
reviews_df['url']=reviews_df['url'].apply(lambda x: x.split('?')[0])
reviews_df.head()

,url,img_url,developer,requirements,name,metadata,clean_description,pegi_url,mature_encoded,all_reviews,clean_review
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,{},Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94...",Overwhelmingly Positive
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,{'minimum': {'windows': {'processor': ' Intel ...,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,0,"16 Oct, 2020","16 Oct, 2020"


In [403]:
smaller_df=smaller_df.rename(columns={'reviews':'clean_review'})
smaller_df

,url,name,developer,tags,languages,genre,game_description,mature_content,price,clean_review,date,achievements,op_sys,metadata,cluster
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,co fps meme classic great sci difficult soundt...,29
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,third co fps massively based battle difficult ...,28
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,rich story great space based sci difficult sou...,3
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,sandbox co open fps massively apocalyptic op a...,38
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,sandbox open space massively sci difficult cap...,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24559,https://store.steampowered.com/app/906490/Game...,Gamers Club,Gamers Club,utility,english portuguese brazil,utility,software largest c go club world platform larg...,0,NaN,no value,EM BREVE,0.0,windows,utility,16
24560,https://store.steampowered.com/app/906470/Gravia/,Gravia,Dan Goyette,action indie,english,action indie,game gravia put power black hole right hand gr...,0,NaN,no value,2021-03-15 00:00:00,0.0,NaN,action indie,8
24561,https://store.steampowered.com/app/906460/Gala...,Galactis,Gustavo Contreras,action indie,english,action indie,game revive glory bygone space shooter arcade ...,0,0.891,no value,2018-08-15 00:00:00,0.0,windows,action indie,2
24562,https://store.steampowered.com/app/906430/Alive/,Alive,OldGHZ,free play strategy action rpg massively multip...,english,action free play massively multiplayer rpg str...,game alive multiplayer third person cover base...,1,NaN,no value,2019-07-11 00:00:00,0.0,NaN,strategy play survival action multiplayer fps ...,14


In [404]:
extra_df=smaller_df[['url','clean_review']].copy()
extra_df.head()

,url,clean_review
0,https://store.steampowered.com/app/379720/DOOM/,Very Positive
1,https://store.steampowered.com/app/578080/PLAY...,Mixed
2,https://store.steampowered.com/app/637090/BATT...,Mostly Positive
3,https://store.steampowered.com/app/221100/DayZ/,Mixed
4,https://store.steampowered.com/app/8500/EVE_On...,Mostly Positive


In [405]:
reviews_df[['clean_review','url']]=reviews_df[['clean_review','url']].astype('str')

In [410]:
extra_df.shape

(24564, 2)

In [415]:
rev_df=reviews_df.merge(extra_df,on=['url','clean_review'], how='left' )

In [416]:
rev_df

,url,img_url,developer,requirements,name,metadata,clean_description,pegi_url,mature_encoded,all_reviews,clean_review
0,https://store.steampowered.com/app/10/CounterS...,https://steamcdn-a.akamaihd.net/steam/apps/10/...,Valve,{},Counter-Strike,First- Shooter Violent+ Score Survival Team- P...,game play world number online action game enga...,NaN,0,"Overwhelmingly Positive(94,680)- 96% of the 94...",Overwhelmingly Positive
1,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
2,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
3,https://store.steampowered.com/app/1000000/ASC...,https://steamcdn-a.akamaihd.net/steam/apps/100...,IndigoBlue Game Studio,{'minimum': {'windows': {'processor': ' Intel ...,ASCENXION,Indie Stick about game Minimalist Controller2 ...,game ascenxion game combining shoot em adventu...,NaN,0,Winter 2020,Winter 2020
4,https://store.steampowered.com/app/1000010/Cro...,https://steamcdn-a.akamaihd.net/steam/apps/100...,NEXT Studios,{'minimum': {'windows': {'processor': ' Intel ...,Crown Trick,Replay Female Support Cards Magic+ Steam Value...,game crown trick beautifully animated rogue li...,NaN,0,"16 Oct, 2020","16 Oct, 2020"
...,...,...,...,...,...,...,...,...,...,...,...
42988,https://store.steampowered.com/app/999860/Enem...,https://steamcdn-a.akamaihd.net/steam/apps/999...,Windwalk Games,{'minimum': {'windows': {'processor': ' Any du...,Enemy On Board,Indie Play agreement Psychological Deduction t...,game enemy board online multiplayer social ded...,NaN,0,"Mostly Positive(1,224)- 77% of the 1,224 user ...",Mostly Positive
42989,https://store.steampowered.com/app/999890/Bruken/,https://cdn.cloudflare.steamstatic.com/steam/a...,FlairBot Games,{'minimum': {'windows': {'processor': ' Intel ...,Bruken,Indie Features Casual Single-playerProfile Adv...,game bruken weird name well get world see piec...,NaN,0,"Apr 11, 2019","Apr 11, 2019"
42990,https://store.steampowered.com/app/999930/Fant...,https://cdn.cloudflare.steamstatic.com/steam/a...,张八万工作室,{'minimum': {'windows': {'processor': ' Intel®...,Fantasy Sino-Japanese War 幻想甲午,Indie Steam Early Profile Features Support Clo...,game unfortunately passed sino japanese war be...,NaN,0,"Jan 11, 2019","Jan 11, 2019"
42991,https://store.steampowered.com/app/999930/Fant...,https://steamcdn-a.akamaihd.net/steam/apps/999...,张八万工作室,{'minimum': {'windows': {'processor': ' Intel®...,Fantasy Sino-Japanese War 幻想甲午,Indie Steam Early Profile Features Support Clo...,game unfortunately passed sino japanese war be...,NaN,0,"11 Jan, 2019","11 Jan, 2019"


In [418]:
rev2_df= rev_df[rev_df['all_reviews'].str.contains("Overwhelmingly Negative|Very Negative|Negative|Mostly Negative|Mixed|Mostly Positive|Positive|Very Positive|Overwhelmingly Positive")]

rev2_df.shape

(4690, 11)

In [420]:
rev_df['clean_review'].isna().sum()

0

In [422]:
rev_df['clean_review'].tail(50)

42943      Very Positive
42944       24 Dec, 2019
42945       Mar 15, 2019
42946       Feb 11, 2019
42947       11 Dec, 2018
42948        Jan 4, 2019
42949       15 Oct, 2020
42950       Mar 21, 2019
42951       26 Dec, 2018
42952       Jul 26, 2019
42953       26 Jul, 2019
42954       28 Dec, 2018
42955       28 Dec, 2018
42956       28 Dec, 2018
42957       Dec 28, 2018
42958       24 Dec, 2018
42959      Very Positive
42960       Dec 12, 2019
42961       Jan 30, 2019
42962      Very Positive
42963    Mostly Positive
42964        Jan 3, 2019
42965       Mar 14, 2019
42966    Mostly Positive
42967        Nov 4, 2019
42968       28 Dec, 2018
42969       Dec 28, 2018
42970       Oct 20, 2020
42971       19 Jan, 2019
42972       Jun 19, 2019
42973       Jun 19, 2019
42974       Jun 19, 2019
42975        6 Dec, 2018
42976       Jan 24, 2019
42977       18 Feb, 2019
42978        Jan 4, 2019
42979        7 Jan, 2019
42980       Mar 26, 2019
42981       22 Nov, 2019
42982        Jan 4, 2019


In [424]:
rev_df.iloc[42972:42975]

,url,img_url,developer,requirements,name,metadata,clean_description,pegi_url,mature_encoded,all_reviews,clean_review
42972,https://store.steampowered.com/app/999270/War_...,https://cdn.cloudflare.steamstatic.com/steam/a...,The Cyber Prophet,{'minimum': {'windows': {'processor': ' Intel ...,War Builder League,Indie Play Shooter Sandbox Robots Massively Pv...,game war builder league free non commercial ho...,NaN,0,"Jun 19, 2019","Jun 19, 2019"
42973,https://store.steampowered.com/app/999270/War_...,https://cdn.cloudflare.steamstatic.com/steam/a...,The Cyber Prophet,{'minimum': {'windows': {'processor': ' Intel ...,War Builder League,Indie Play Shooter Sandbox Robots Massively Pv...,game war builder league free non commercial ho...,NaN,0,"Jun 19, 2019","Jun 19, 2019"
42974,https://store.steampowered.com/app/999270/War_...,https://cdn.cloudflare.steamstatic.com/steam/a...,The Cyber Prophet,{'minimum': {'windows': {'processor': ' Intel ...,War Builder League,Indie Play Shooter Sandbox Robots Massively Pv...,game war builder league free non commercial ho...,NaN,0,"Jun 19, 2019","Jun 19, 2019"


In [425]:
rev_df.shape

(42993, 11)

In [434]:
rev3_df=rev_df.drop_duplicates(subset='url')
rev3_df.shape

(36412, 11)

In [436]:
rev3_df=rev3_df.rename({'clean_review':'reviews','mature_encoded':'mature_content','clean_description':'game_description'})

In [437]:
rev3_df.to_csv("../raw_data/clean_bigger_df.csv")